In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
import evaluate
from transformers import AutoTokenizer, AutoModelForTokenClassification
import functions


seqeval = evaluate.load("seqeval")


In [3]:
#@ Data Preprocessing

import pandas as pd
import ast 
df = pd.read_csv('data.csv')
df['labels'] = df['labels'].apply(ast.literal_eval)

In [4]:
labels = [i for i in df['labels'].values.tolist()]
unique_labels = set()

for lb in labels:
        [unique_labels.add(i) for i in lb if i not in unique_labels]
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

In [5]:
#@ Model Construction 
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [8]:
# Load model directly from huggingface
model = BertModel()
tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")

In [79]:
#@ Model Training

from train_evaluate import train_loop
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
model = BertModel()
train_loop(model, tokenizer,  df_train, df_val)

100%|██████████| 106/106 [00:25<00:00,  4.12it/s]


Epochs: 1 | Loss:  1.333 | Accuracy:  0.537 | Val_Loss:  0.893 | Accuracy:  0.665
Saved new best model with accuracy  0.665


100%|██████████| 106/106 [00:25<00:00,  4.14it/s]


Epochs: 2 | Loss:  0.716 | Accuracy:  0.767 | Val_Loss:  0.616 | Accuracy:  0.801
Saved new best model with accuracy  0.801


100%|██████████| 106/106 [00:26<00:00,  4.04it/s]


...

Epochs: 8 | Loss:  0.168 | Accuracy:  0.950 | Val_Loss:  0.429 | Accuracy:  0.881


100%|██████████| 106/106 [00:24<00:00,  4.31it/s]


Epochs: 9 | Loss:  0.126 | Accuracy:  0.961 | Val_Loss:  0.408 | Accuracy:  0.886


100%|██████████| 106/106 [00:25<00:00,  4.21it/s]


Epochs: 10 | Loss:  0.101 | Accuracy:  0.972 | Val_Loss:  0.373 | Accuracy:  0.904


In [10]:
#@ Model Evaluation on Test Set
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

BertModel(
  (bert): DistilBertForTokenClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN

In [12]:
from evaluate_func import evaluate
evaluate(model, df_test, tokenizer)

Test Accuracy:  0.868
